### Supabase REST API

This file handles user registration and login requests using FastAPI and Supabase.

It uses the Supabase REST API to interact with the user database.

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import requests

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()  # Load .env file
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
url = f"{SUPABASE_URL}/rest/v1/users"

In [ ]:
app = FastAPI()

# Define input schema
class UserCreate(BaseModel):
    username: str
    email: str
    password: str  # Password should be hashed before storing

In [ ]:
@app.get("/")
def root():
    return {"message": "Welcome to the AI Fitness API"}

In [ ]:
# Endpoint to register a new user
@app.post("/register")
def register_user(user: UserCreate):
    headers = {
        "apikey": SUPABASE_KEY,
        "Authorization": f"Bearer {SUPABASE_KEY}",
        "Content-Type": "application/json"
    }

    # Validate that the user does not already exist
    payload = user.dict()

    response = requests.post(url, headers=headers, json=payload)

    if response.status_code != 201:
        return {
            "status": "error",
            "code": response.status_code,
            "detail": response.text
        }
    return {"message": "User added successfully!"}

### Hash Password Before Saving

#### Hashing vs Enryption

| Feature                      | Hashing (`bcrypt`)     | Encryption                   |
| ---------------------------- | ---------------------- | ---------------------------- |
| **One-way?**                 | ✅ Yes (irreversible)   | ❌ No (can be reversed)       |
| **Use case**                 | Password storage       | Message secrecy              |
| **Needs secret key?**        | ❌ No                   | ✅ Yes (to decrypt)           |
| **Same input, same output?** | ❌ bcrypt adds a "salt" | ✅ Usually, unless randomized |


#### How ```verify()``` works

| Step | What happens                                |
| ---- | ------------------------------------------- |
| 1    | Parse salt & cost from stored hash          |
| 2    | Re-hash user’s input using same salt & cost |
| 3    | Compare both hashes securely                |
| ✅    | If match → allow login                      |


In [1]:
from passlib.context import CryptContext

In [2]:
# Set up hashing algorithm
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

def hash_password(password: str) -> str:
    return pwd_context.hash(password)

In [3]:
hash = hash_password("my_secure_password")
print(f"Hashed password: {hash}")

(trapped) error reading bcrypt version
Traceback (most recent call last):
  File "/workspaces/Rep-BodAI/.venv/lib/python3.12/site-packages/passlib/handlers/bcrypt.py", line 620, in _load_backend_mixin
    version = _bcrypt.__about__.__version__
              ^^^^^^^^^^^^^^^^^
AttributeError: module 'bcrypt' has no attribute '__about__'


Hashed password: $2b$12$wtkhoA8jurHp4S49Oy9pmuGCQuGEg7o1nUjspoaAZUIsaKdn9LUH2


In [4]:
pwd_context.verify("my_secure_password", hash)  # returns True


True